In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
base_url=requests.get("https://ca.indeed.com/cmp/Google/reviews?start=",timeout=100)

In [3]:
print(base_url.text)


<!doctype html><html><head><title>Working at Google: 65 Reviews | Indeed.com</title><meta http-equiv="content-type" content="text/html;charset=UTF-8"><meta name="description" content="65 reviews from Google employees about Google culture, salaries, benefits, work-life balance, management, job security, and more."><meta name="viewport" content="width=device-width, initial-scale=1"><link rel="canonical" href="https://ca.indeed.com/cmp/Google/reviews"><link rel="next" href="https://ca.indeed.com/cmp/Google/reviews?start=20"><link rel="alternate" href="https://ar.indeed.com/cmp/Google/reviews" hreflang="es-AR"><link rel="alternate" href="https://at.indeed.com/cmp/Google/reviews" hreflang="de-AT"><link rel="alternate" href="https://au.indeed.com/cmp/Google/reviews" hreflang="en-AU"><link rel="alternate" href="https://be.indeed.com/cmp/Google/reviews" hreflang="nl-BE"><link rel="alternate" href="https://emplois.be.indeed.com/cmp/Google/reviews" hreflang="fr-BE"><link rel="alternate" href="ht

In [4]:
def parse(full_url):
    page_content =  BeautifulSoup(full_url.content, 'html.parser')    containers = page_content.findAll('div',{'class':'cmp-Review-container'})
    print(containers)
    df = pd.DataFrame(columns = ['rating', 'rating_tile' ,'rating_description', 'rating_pros', 'rating_cons'])

SyntaxError: invalid syntax (<ipython-input-4-327318f7abfa>, line 2)

In [9]:
def parse(full_url):
   # page_content =  BeautifulSoup(full_url.context,'lxml')
    page_content = BeautifulSoup(full_url.content, 'html.parser')
    containers = page_content.findAll('div',{'class':'cmp-Review-container'})
    #print(containers)
    df = pd.DataFrame(columns = ['rating', 'rating_title' ,'rating_description','rating_pros', 'rating_cons'])

    for item in containers:
        try:
            rating = item.find('div', {'class' : 'cmp-ReviewRating-text'}).text.replace('\n', '')
        except:
            rating = None
        try:
            rating_title = item.find('div', {'class' : 'cmp-Review-title'}).text.replace('\n', '')
        except:
            rating_title = None
        try:
            rating_description = item.find('div', {'class' : 'cmp-Review-text'}).text.replace('\n', '')
        except:
            rating_description = None
        try:
            rating_pros = item.find('div', {'class' : 'cmp-ReviewProsCons-prosText'}).text.replace('\n', '')
        except:
            rating_pros = None
        try:
            rating_cons = item.find('div', {'class' : 'cmp-ReviewProsCons-consText'}).text.replace('\n', '')
        except:
            rating_cons = None
        df = df.append({'rating':rating,'rating_title':rating_title,'rating_description':rating_description,'rating_pros':rating_pros,'rating_cons':rating_cons},ignore_index=True)
    return df

In [38]:
base_url="https://ca.indeed.com/cmp/Google/reviews?start="
all_reviews_df = pd.DataFrame(columns = ['rating', 'rating_title' ,'rating_description','rating_pros', 'rating_cons'])
num_reviews=20

while num_reviews < 4000:
    full_url=base_url + str(num_reviews)
    get_url = requests.get(full_url,timeout=5)
    partial_df=parse(get_url)
    all_reviews_df = all_reviews_df.append(partial_df,ignore_index=True)
    num_reviews+=20

In [39]:
display(all_reviews_df)

,rating,rating_title,rating_description,rating_pros,rating_cons
0,5.0,Technical Support Mississauga Ontario,Customer service/call centre mainly tech suppo...,None,None
1,5.0,Technical Support Mississauga Ontario,Customer service/call centre mainly tech suppo...,None,None
2,4.0,great place to work,"the management is great, the culture is awesom...","managment, people, culture",money
3,2.0,Lots of politics,The work culture has changed so that there is ...,Free lunches,Unfulfilling work
4,3.0,Productive company,A typical day at work in the company is fast p...,None,None
...,...,...,...,...,...
4137,5.0,Line Cook Working Omelette station during brea...,"Truly a great place to work, Chef was an aweso...",Free food,Early start time?
4138,5.0,Fun place,I am glad today that I login here.I like it ...,Translation and writing,Healhacare
4139,3.0,Google should know better,Google shouldn't participate in project Dragon...,None,None
4140,5.0,Fun place!,Google is an awesome place to work! The hours ...,Collaborative,Long Hours


In [40]:
all_reviews_df.to_csv("Indeed_scrape.csv")